<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import warnings
warnings.filterwarnings("ignore")

/home/jdowling/anaconda3/envs/book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get AQI API Token

Go here first, to get your AQI API key
https://aqicn.org/data-platform/token/

Then enter the token you copied into the following variable in the code below. 

AQI_API_KEY = 

In [2]:
# Getting the current date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

country="sweden"
city="stockholm"
street="stockholm-hornsgatan-108-gata"

project = hopsworks.login()
fs = project.get_feature_store() 

# Get your AQI_API_KEY from here:
# https://aqicn.org/data-platform/token/

#AQI_API_KEY = # PUT YOUR AQI API KEY HERE
AQI_API_KEY = hopsworks.connection().get_secrets_api().get_secret("AQI_API_KEY").value


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works:443/p/5240
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [3]:

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Get Yesterday's Air Quality data (PM2.5) from the Feature Group</span>
My Air quality sensor is:  https://aqicn.org/station/sweden/stockholm-sodermalm/ 

In [4]:
# Read all data from the feature group
df_air_quality = air_quality_fg.read()

df = pd.DataFrame()
df_air_quality['day'] = pd.to_datetime(df_air_quality['date']).dt.date

Finished: Reading data from Hopsworks, using Hive (2.78s) 


In [5]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city", "street"]].groupby("street").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

In [8]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date
0,11.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2024-03-09


In [9]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float32(1), object(3)
memory usage: 164.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather data</span>

In [10]:
hourly_df = util.get_hourly_weather_forecast(city)
hourly_df = hourly_df.set_index('date')

## We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.19999694824219°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-03-09,0.25,0.0,10.002560,59.743652,stockholm
1,2024-03-10,0.95,0.0,15.790833,114.227737,stockholm
2,2024-03-11,3.00,0.0,12.864649,107.928009,stockholm
3,2024-03-12,5.55,0.0,7.754637,111.801476,stockholm
4,2024-03-13,5.30,0.0,2.099143,30.963696,stockholm
5,2024-03-14,0.70,0.1,16.167967,348.439911,stockholm
6,2024-03-15,3.95,0.0,6.696387,53.746078,stockholm
7,2024-03-16,3.30,0.0,9.114471,189.090195,stockholm
8,2024-03-17,6.55,0.0,7.895416,114.227737,stockholm
9,2024-03-18,6.45,0.0,11.542478,86.423744,stockholm


In [11]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
